In [1]:
import inspire_agrivolt
from glob import glob
import xarray as xr
import pathlib
import subprocess

from pathlib import Path
import pvdeg
import json

In [ ]:
v1 = Path("/projects/inspire/PySAM-MAPS/v1")

states = [
    "alabama",
    "arizona",
    "arkansas",
    "california",
    "florida",
    "georgia",
    "hawaii",
    "louisiana",
    "mississippi",
    "new_mexico",
    "oklahoma",
    "south_carolina",
    "texas",
]

confs = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",

]

bad = {}

for state in states:
    for conf in confs:
    
        gcr_fixed = v1 / Path(f"{state}/{conf}")
        print(state, conf)
    
        for path in gcr_fixed.iterdir():
            try:
                xr.open_zarr(path)
            except Exception as e:
                bad[str(path)] = e
                continue

In [ ]:
gcr_fixed = v1 / Path(f"texas/01")

xr.open_zarr(next(gcr_fixed.iterdir()))


In [ ]:
bad

In [ ]:
WEATHER_DB = "NSRDB"
WEATHER_ARG = {
    "satellite": "Americas",
    "names": "TMY",
    "NREL_HPC": True,
    "attributes": pvdeg.pysam.INSPIRE_NSRDB_ATTRIBUTES,
}

geo_weather, geo_meta = pvdeg.weather.get(
    WEATHER_DB, geospatial=True, **WEATHER_ARG
)

In [ ]:
zarrs = {}

for path in Path('/projects/inspire/PySAM-MAPS/v1/hawaii').iterdir():

    outs = path.glob("*.zarr")

    res = []
    for out in outs:
        res.append(xr.open_zarr(str(out)))


    zarrs[str(path)] = pvdeg.utilities.gids_dataset_to_coords_dataset( # convert to lat long dataset
        xr.concat(res, dim="gid"), # load gid datasets
        geo_meta
    )

In [ ]:
zarrs['/projects/inspire/PySAM-MAPS/v1/hawaii/09'].pitch.plot()

In [ ]:
res = [xr.open_zarr(str(p)) for p in Path('/projects/inspire/PySAM-MAPS/v1/model-outs/florida/09/').glob("*.zarr")]


out = pvdeg.utilities.gids_dataset_to_coords_dataset( # convert to lat long dataset
    xr.concat(res, dim="gid"), # load gid datasets
    geo_meta
)

out.pitch.plot()

In [ ]:
pvdeg.geospatial.start_dask()

In [ ]:
states = ["alabama",
"arizona",
"arkansas",
"california",
"colorado",
"connecticut",
"delaware",
"florida",
"georgia",
"idaho",
"illinois",
"indiana",
"iowa",
"kansas",
"kentucky",
"louisiana",
"maine",
"maryland",
"massachusetts",
"michigan",
"minnesota",
"mississippi",
"missouri",
"montana",
"nebraska",
"nevada",
"new_hampshire",
"new_jersey",
"new_mexico",
"new_york",
"north_carolina",
"north_dakota",
"ohio",
"oklahoma",
"oregon",
"pennsylvania",
"rhode_island",
"south_carolina",
"south_dakota",
"tennessee",
"texas",
"utah",
"vermont",
"virginia",
"washington",
"west_virginia",
"wisconsin",
"wyoming",]

confs = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]

In [ ]:
results = {}

for state in states:
    for conf in confs:
        print(state, conf)
    
        MODEL_OUT_PATH = Path(f'/projects/inspire/PySAM-MAPS/v1/{state}/{conf}')
        MODEL_OUT_ZARRS = list(MODEL_OUT_PATH.glob("*.zarr"))
        
        zarrs = [xr.open_zarr(str(path)) for path in MODEL_OUT_ZARRS]
        joined = xr.concat(zarrs, dim="gid").sortby("gid")
    
        gids, breakdown = inspire_agrivolt.verify.gids_with_time_nans(
            joined
        )
    
        if state not in results:
            results[state] = {}
    
        results[state][conf] = breakdown



with open("breakdown.json", "w") as fp:
    fp.write(json.dumps(results))

# joined

In [ ]:
with open("/projects/inspire/PySAM-MAPS/v1/model-outs/completeness.json", "r") as fp:
    results = json.load(fp)

fields = [
    "albedo",
    "dhi",
    "dni",
    "ghi",
    "ground_irradiance",
    "relative_humidity",
    "subarray1_celltemp",
    "subarray1_dc_gross",
    "subarray1_poa_front",
    "subarray1_poa_rear",
    "temp_air",
    "wind_direction",
    "wind_speed",
]
confs = [
    "01",
    "02",
    "03",
    "04",
    "05",
    "06",
    "07",
    "08",
    "09",
    "10",
]

for state in results:
    for conf in confs:
        stats = results[state][conf]

        for field in fields:
            if stats[field] != 0:
                print(state, conf, "found non-zero missing gids")


In [ ]:
gids, breakdown = inspire_agrivolt.verify.gids_with_time_nans(
    joined
)

In [ ]:
breakdown

In [ ]:
inspire_agrivolt.verify.summarize_nans_nd(
    joined
)